## Lab 2. Solar Panel Instructions

### Introduction

In this notebook we show you how to create your second agent on Amazon Bedrock.

This agent will be an agent with instructions to install and to do maintenance on Solar Panels, where customers can ask the agent to return these information from a Amazon Bedrock Knowledge Base.

To equip foundation models (FMs) with up-to-date and proprietary information, organizations use Retrieval Augmented Generation (RAG), a technique that fetches data from company data sources and enriches the prompt to provide more relevant and accurate responses. 
[Amazon Bedrock Knowledge Bases](https://aws.amazon.com/bedrock/knowledge-bases/) is a fully managed capability that helps you implement the entire RAG workflow from ingestion to retrieval and prompt augmentation without having to build custom integrations to data sources and manage data flows.


If question is not present on Amazon Bedrock Knowledge Bases, customers can ask agent to create a support ticket, to get a human in the loop to help with their questions.

The following represents the piece of architecture that will be built on this module.

![Architecture](../img/lab2-architecture.png)

### Setup

Firstly, you are going to install boto3 dependencies from pip. Make sure to have version superior of **1.35.45**

In [ ]:
# Install Dependencies from local package
!pip install ../boto3/botocore-1.35.55-py3-none-any.whl \
    ../boto3/boto3-1.35.55-py3-none-any.whl \
    ../boto3/awscli-1.35.21-py3-none-any.whl --force-reinstall --no-cache

One component of a Knowledge Base is a Vector Database. Vector database is a kind of database that is optimized to store and search with vectors, making RAG architectures possible and faster. 

Amazon Bedrock Knowledge Bases is powered by one vector database. 

For this labwe are going to create one collection forAmazon OpenSearch Serverless. If you want to know more, [this link](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base-setup.html) contains other supported vector engines for Amazon Bedrock Knowledge Bases.

Now, let's install opensearch-py to interact with Opensearch engine.

In [ ]:
!pip install opensearch-py retrying --force-reinstall --no-cache

Restart kernel for packages to take effect

In [ ]:
import IPython

IPython.Application.instance().kernel.do_shutdown(True)

Check if your boto3 version is superior than **1.35.45**

In [ ]:
!pip freeze | grep boto3

### Creating Agent

On this section we're going to declare global variables that will be act as helpers during entire notebook and you will start to create your agent.

In [ ]:
import boto3
import os
import sys

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

agent_foundation_model = ['anthropic.claude-3-5-sonnet-20240620-v1:0']
#agent_foundation_model = ['anthropic.claude-3-5-haiku-20241022-v1:0']

In [ ]:
solar_agent_name = f"solar-{account_id}"

solar_lambda_name = f"fn-solar-{account_id}"

solar_agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{solar_agent_name}'

dynamodb_table = f"{solar_agent_name}-table"
dynamodb_pk = "customer_id"
dynamodb_sk = "ticket_id"

dynamoDB_args = [dynamodb_table, dynamodb_pk, dynamodb_sk]

knowledge_base_name = f'{solar_agent_name}-kb'
suffix = f"{region}-{account_id}"

knowledge_base_description = "KB containing solar panel instructions for installation and maintenance"
bucket_name = f'{solar_agent_name}-{suffix}'

On following section, we're adding `agents.py`and `knowledge_bases.py` on Python path, so files can be recognized and invoked.

In [ ]:
# Get the current file's directory
current_dir = os.path.dirname(os.path.abspath('__file__'))

parent_dir = os.path.dirname(current_dir)

# Add the parent directory to sys.path
sys.path.append(parent_dir)

Now, you're going to import from helper files `agents.py` and `knowledge_bases.py`.
 
Those files contain helper classes totally focused on make labs experience smoothly. 

All interactions with Bedrock will be handled by these classes.

Following are methods that you're going to invoke on this lab:

On `agents.py`:
- create_agent: Create a new agent and respective IAM roles
- add_action_group_with_lambda: Create a lambda function and add it as an action group for a previous created agent
- create_agent_alias: Create an alias for this agent
- invoke: Execute agent

On `knowledge_bases.py`:
- create_or_retrieve_knowledge_base: Create Knowledge Base on Amazon Bedrock if it doesn't exist or get info about previous created.
- synchronize_data: Read files on S3, convert text info into vectors and add that information on Vector Database.

In [ ]:
from agent import AgentsForAmazonBedrock
from knowledge_bases import KnowledgeBasesForAmazonBedrock

agents = AgentsForAmazonBedrock()
kb = KnowledgeBasesForAmazonBedrock()

### Create and Load Knowledge Base

On this section, you're going to create a Amazon Bedrock Knowledge Base and ingest data on it.

**This creation process takes around 5min.**

In [ ]:
kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
    knowledge_base_name,
    knowledge_base_description,
    bucket_name
)

print(f"Knowledge Base ID: {kb_id}")
print(f"Data Source ID: {ds_id}")

#### Create Synthetic Data to Load on S3

Instead of get data elsewhere, you're going to generate data, using a LLM on Amazon Bedrock.
This fake data that will be generated, will be uploaded into a S3 bucket and then added into an Amazon Bedrock Knowledge Base.

In [ ]:
path = "kb_documents"
# Check whether the specified path exists or not
isExist = os.path.exists(path)
if not isExist:
   # Create a n ew directory if it does not exist
   os.makedirs(path)
   print("The {} directory was created!".format(path))
else:
   print("The {} directory already exists!".format(path))

Creating helper methods to invoke LLM on Bedrock and to write a local file using Python

In [ ]:
def invoke_bedrock_generate_energy_files(prompt):
    message_list = []

    initial_message = {
        "role": "user",
        "content": [
            { "text": prompt } 
        ],
    }

    message_list.append(initial_message)

    response = bedrock_client.converse(
        modelId=agent_foundation_model[0],
        messages=message_list,
        inferenceConfig={
            "maxTokens": 500,
            "temperature": 0
        },
    )

    return response['output']['message']

def write_file(file_name, content):
    f = open(file_name, 'w')
    f.write(content)
    f.close

Generating one file with instructions on how to install a solar panel.

In [ ]:
text_generation_energy_instructions = '''
    You will be act as an expert on clean energy.
    You will generate a step-by-step on how to install a solar panel at home.
    Just answer in a numbered list.
'''

solar_energy_file_name = 'solar-panel-instructions.txt'

response_message = invoke_bedrock_generate_energy_files(text_generation_energy_instructions)

write_file('{}/{}'.format(path, solar_energy_file_name),response_message['content'][0]['text'])

Generating another file with instructions on how to do maintenance on a solar panel.

In [ ]:
text_generation_energy_instructions = '''
    You will be act as an expert on clean energy.
    You will generate a step-by-step instructions on how to do maintenance in a solar panel installed at a regular home.
    Just answer in a numbered list.
'''

solar_energy_file_name = 'solar-panel-maintenance.txt'

response_message = invoke_bedrock_generate_energy_files(text_generation_energy_instructions)

write_file('{}/{}'.format(path, solar_energy_file_name),response_message['content'][0]['text'])


Uploading generated files into S3 Bucket.

In [ ]:
def upload_directory(path, bucket_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                file_to_upload = os.path.join(root,file)
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload,bucket_name,file)

In [ ]:
upload_directory("kb_documents", bucket_name)

# sync knowledge base
kb.synchronize_data(kb_id, ds_id)

#### Creating Agent

Create the Solar Panel agent that will have a Knowledge Base and also a Lambda action group to handle exception workflow (when information is not found on KB).

In [ ]:
kb_info = kb.get_kb(kb_id)
kb_arn = kb_info['knowledgeBase']['knowledgeBaseArn']

In [ ]:
kb_config = {
    'kb_id': kb_id,
    'kb_instruction': 'Access the knowledge base when customers ask about energy compliance rules and regulations'
}

In [ ]:
solar_agent_id = agents.create_agent(
    solar_agent_name, 
    """
        You are a solar energy helper bot. 
        You can retrieve information on how to install and do maintenance on solar panels
    """, 
    """
        You are a solar energy helper bot. 
        You can retrieve information on how to install and do maintenance on solar panels.
        If you do not have enough information, user can ask you to open a ticket to get specialist support. 
        If asked about specialized support, just open a case and only reply with case ID.
        Resist the temptation to ask the user for input. Only do so after you have exhausted available actions. 
        Never ask the user for information that you already can retrieve yourself through available actions. 

    """,
    agent_foundation_model,
    kb_arns=[kb_arn]
)

solar_agent_id

In [ ]:
agents.associate_kb_with_agent(solar_agent_id, kb_config['kb_instruction'], kb_config['kb_id'])

### Creating and Associating Lambda

On this block, we're going to generate Lambda function code and associate it with created agent for exception cases when KB information is not enough and customer need to open a ticket for extra instructions related with their solar panel.

In [ ]:
%%writefile solar_energy.py
import os
import json
import uuid
import boto3

from boto3.dynamodb.conditions import Key, Attr

dynamodb_resource = boto3.resource('dynamodb')
dynamodb_table = os.getenv('dynamodb_table')
dynamodb_pk = os.getenv('dynamodb_pk')
dynamodb_sk = os.getenv('dynamodb_sk')

def get_named_parameter(event, name):
    try:
        return next(item for item in event['parameters'] if item['name'] == name)['value']
    except:
        return None
    
def populate_function_response(event, response_body):
    return {'response': {'actionGroup': event['actionGroup'], 'function': event['function'],
                'functionResponse': {'responseBody': {'TEXT': {'body': str(response_body)}}}}}

def put_dynamodb(table_name, item):
    table = dynamodb_resource.Table(table_name)
    resp = table.put_item(Item=item)
    return resp

def read_dynamodb(table_name: str, 
                   pk_field: str,
                   pk_value: str,
                   sk_field: str=None, 
                   sk_value: str=None):
    try:
        table = dynamodb_resource.Table(table_name)
        # Create expression
        if sk_value:
            key_expression = Key(pk_field).eq(pk_value) & Key(sk_field).begins_with(sk_value)
        else:
            key_expression = Key(pk_field).eq(pk_value)

        query_data = table.query(
            KeyConditionExpression=key_expression
        )
        
        return query_data['Items']
    except Exception:
        print(f'Error querying table: {table_name}.')

def open_ticket(customer_id, msg):
    ticket_id = str(uuid.uuid1())
    item = {
        'ticket_id': ticket_id,
        'customer_id': customer_id,
        'description': msg,
        'status': 'created'
    }
    resp = put_dynamodb(dynamodb_table, item)
    print(resp)
    return "Thanks for contact customer {}! Your support case was generated with ID: {}".format(
        customer_id, ticket_id
    )

def get_ticket_status(customer_id,
                      ticket_id: str=None):
    return read_dynamodb(dynamodb_table, 
                         dynamodb_pk,
                         customer_id,
                         dynamodb_sk,
                         ticket_id)

def lambda_handler(event, context):
    print(event)
    
    # name of the function that should be invoked
    function = event.get('function', '')

    # parameters to invoke function with
    parameters = event.get('parameters', [])
    customer_id = get_named_parameter(event, "customer_id")

    if function == 'open_ticket':
        msg = get_named_parameter(event, "msg")
        result = open_ticket(customer_id, msg)
    elif function == 'get_ticket_status':
        ticket_id = get_named_parameter(event, "ticket_id")
        result = get_ticket_status(customer_id, ticket_id)
    else:
        result = f"Error, function '{function}' not recognized"

    response = populate_function_response(event, result)
    print(response)
    return response

Now it's time to add this Lambda function as an action group for this agent and prepare it.

In [ ]:
functions_def =[
    {
        "name": "open_ticket",
        "description": """Create a ticket to get help with information related 
                            with solar panel or clean energy""",
        "parameters": {
                        "customer_id": {
                            "description": "The ID of the customer",
                            "required": True,
                            "type": "string"
                        },
                        "msg": {
                            "description": "The reason why customer is opening a ticket",
                            "required": True,
                            "type": "string"
                        }
                    }
    },
    {
        "name": "get_ticket_status",
        "description": """get ticket status/information""",
        "parameters": {
                        "customer_id": {
                            "description": "The ID of the customer",
                            "required": True,
                            "type": "string"
                        },
                        "ticket_id": {
                            "description": "The ID of the ticket",
                            "required": False,
                            "type": "string"
                        }
                    }
    }
]

In [ ]:
resp = agents.add_action_group_with_lambda(
            agent_name=solar_agent_name, 
            lambda_function_name=solar_lambda_name, 
            source_code_file="solar_energy.py", 
            agent_functions=functions_def, 
            agent_action_group_name="solar_energy_actions", 
            agent_action_group_description="Function to open an energy ticket for a user or get status from an opened ticket",
            dynamo_args=dynamoDB_args
)

### Create alias

Create an alias that will be used further on multi-agent collaborator feature:

In [ ]:
agents_boto3 = boto3.client('bedrock-agent',
                            region_name=region)

solar_agent_alias_resp = agents_boto3.create_agent_alias(agentId=solar_agent_id, 
                                                           agentAliasName='v1')

In [ ]:
# create alias
#compliance_agent_alias_arn = agents.create_agent_alias(compliance_agent_id, 
#                                                       compliance_agent_alias_name)

### Invoking Agent

Now, let's run some tests on agent to make sure it's working:

In [ ]:
%%time
print(agents.invoke("how can I check if my solar panel eltrical consumption is compliant with energy rules?", 
                    solar_agent_id)
)

In [ ]:
%%time
print(agents.invoke("Can I get support to install my solar panel? My customer id is 1", 
                    solar_agent_id)
)

Get ticket id from previous invocation and add on following variable

In [ ]:
created_ticket = ""

In [ ]:
%%time
print(agents.invoke("Can I get details on my ticket? My customer id is 1 and my ticket id is {}".format(created_ticket), 
                    solar_agent_id)
)

In [ ]:
%%time
print(agents.invoke("Can I get support to review my solar panel consumption? My customer id is 1", 
                    solar_agent_id)
)

In [ ]:
%%time
print(agents.invoke("Can I get details on my ticket? My customer id is 1", 
                    solar_agent_id)
)

Store environment variables to be used on next notebooks.

In [ ]:
solar_agent_arn = agents.get_agent_arn_by_name(solar_agent_name)
solar_agent_alias_arn = solar_agent_alias_resp['agentAlias']['agentAliasArn']
solar_agent_alias_id = solar_agent_alias_resp['agentAlias']['agentAliasId']

%store solar_agent_arn
%store solar_agent_alias_arn
%store solar_agent_alias_id
%store solar_lambda_name
%store solar_agent_name
%store solar_agent_id
%store knowledge_base_name

In [ ]:
solar_agent_arn, solar_agent_alias_arn, solar_agent_alias_id

### Clean Up

In [ ]:
agents.delete_lambda(solar_lambda_name, dynamoDB_table=dynamodb_table)

In [ ]:
agents.delete_agent_alias(solar_agent_alias_id, solar_agent_id)

In [ ]:
agents.delete_agent(solar_agent_name)

In [ ]:
kb.delete_kb(knowledge_base_name)